# Lead molecule conformation searching in aquesou solution
-Input a SMILES string
- Click run all cells from top left Run menu
- The logic behind this, is we will use GAFF force field to parameterize the organic molecule, then a Newtonian dynamics of this small molecule and thousands of surronding water/ion molecules, will be simulated all together, to finally give a stable conformation for this small molecule during the interaciton with its aqueous surrounding.
- This method is more reliable in drug development project since it is more close to the real in vivo cell solution, compared to some other vacuum mehtod having no solvent involved at all.

In [200]:
import BioSimSpace as BSS

In [ ]:
organic = BSS.Parameters.gaff("c1ccccc1").getMolecule()

# Setting up

In [ ]:
box, angles = BSS.Box.truncatedOctahedron(10*BSS.Units.Length.nanometer)
print(box, angles)

In [ ]:
box_min, box_max = organic.getAxisAlignedBoundingBox()
box_size = [y - x for x, y in zip(box_min, box_max)]
padding = 15 * BSS.Units.Length.angstrom
box_length = max(box_size) + 2*padding

In [ ]:
solvated = BSS.Solvent.tip3p(molecule=organic, box=3*[box_length])

In [ ]:
print(f"solvated = {solvated.charge()}, organic = {organic.charge()}")

# View

In [ ]:
view = BSS.Notebook.View(solvated)
view.system()

# Minimization

In [ ]:
BSS.IO.saveMolecules("solvated", solvated, ["prm7", "rst7"])

In [ ]:
BSS.Process.engines()

In [ ]:
system = BSS.IO.readMolecules("./solvated*")

In [ ]:
protocol = BSS.Protocol.Minimisation(steps=1000)

In [ ]:
process = BSS.Process.OpenMM(system, protocol)

In [ ]:
process.start()

In [ ]:
process.isRunning()

In [ ]:
process.runTime()

# Rfresh the system defination & NVT Equilibration

In [ ]:
minimised = process.getSystem()

In [ ]:
#protocol = BSS.Protocol.Equilibration()
#print(protocol)

In [ ]:
protocol = BSS.Protocol.Equilibration(runtime=5*BSS.Units.Time.picosecond,
                                      temperature_start=0*BSS.Units.Temperature.kelvin,
                                      temperature_end=300*BSS.Units.Temperature.kelvin,
                                      restraint=None)

In [ ]:
process = BSS.Process.OpenMM(minimised, protocol)

In [ ]:
process.inputFiles()

In [ ]:
process.start()

In [ ]:
process.isRunning()

In [ ]:
process.runTime()

# NVT progress
- A total time of 20 ps is designed 

In [ ]:
plot1 = BSS.Notebook.plot(process.getTime(time_series=True), process.getTemperature(time_series=True))

In [ ]:
plot = BSS.Notebook.plot(process.getTime(time_series=True), process.getTotalEnergy(time_series=True))

## Rfresh the system defination & NPT Equilibration

In [ ]:
nvt= process.getSystem(block='True')

# NPT equlibration

In [ ]:
#protocol = BSS.Protocol.Equilibration(runtime=5*BSS.Units.Time.nanosecond,
#                                     pressure=BSS.Units.Pressure.atm,
#                                     report_interval=2500,
#                                     restart_interval=25000,
#                                     restraint=None)

In [ ]:
#process = BSS.Process.OpenMM(nvt, protocol)

In [ ]:
#process.start()

# NPT progress

# Save result
- The file named as simulated.pdb is the file you need download
- This simulated.pdb has the conformation that this molecule propbably to adopt in a aqueous solution

In [ ]:
view.savePDB("simulated.pdb", index=0)

In [ ]:
#npt= process.getSystem()

In [ ]:
view = BSS.Notebook.View(process)

# Visualization

In [ ]:
view.molecule(1)